In [11]:
# import packages
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch
import requests
import os
from os import listdir
import json

In [12]:
# load model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

/ext3/miniconda3/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [13]:
root_dir = "yars_data/"
def gather_photos(label: str, no_captions: bool = False):
    """
    takes the label (category of images to fetch) and filters photos.json for images with that label
    filters for images that 
    returns: list of photo objects (photo_id, label, caption, etc.)
    """ 
    with open(os.path.join(root_dir, 'photos.json'), 'r') as file:
        all_photos = []
        for line in file:
            line = line.rstrip()
            try:
                photo_record = json.loads(line)
                # filers for only ones wiht captions by default
                has_captions = len(photo_record['caption']) > 0 or no_captions 
                if photo_record['label'] == label and has_captions:
                    all_photos.append(photo_record["photo_id"] + ".jpg")
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON: {e} at line: {line}")
        return all_photos
labeled_test = gather_photos('food', no_captions = False)[:100]
unlabeled_test = gather_photos('food', no_captions = True)[:100]

In [14]:
def divide_chunks(l, n): 
    # looping till length l 
    for i in range(0, len(l), n):  
        yield l[i:i + n] 

In [15]:
# if running for the first time

# generating test set feature extraction for 100 labeled images
image_dir = root_dir + "photos/"
chunk_size = 20
chunks = divide_chunks(labeled_test, chunk_size)

paths  = []
failed = []
features = torch.zeros(1, 768)
for chunk in chunks:
    images = []
    for path in chunk:
        try:
            images.append(Image.open(image_dir + path))
            paths.append(path)
            print("\rLoading image %d of %d" % (len(paths) + len(failed), len(labeled_test)), end="")
        except Exception as e:
            print(e)
            print("\r                                  - Failed to load image %s" % image_dir + path, end="")
            failed.append(path)
    print("\rProcessing image %d of %d" % (len(paths) + len(failed), len(labeled_test)), end="")
    inputs = processor(images=images, return_tensors="pt")
    
    image_features = model.get_image_features(**inputs)
    features = torch.cat((features, image_features), dim=0)

    # write data to file
    path_out    = "test/caption_paths.txt"
    feature_out = "test/caption_features.pt" # this is a binary file, shape is (n_images, 768)
    
    with open(path_out, "w") as f:
        for path in paths:
            f.write(path + "\n")
    
    with open(feature_out, "wb") as f:
        torch.save(features, f)

Processing image 100 of 100

In [16]:
# generating test set feature extraction for 100 unlabeled images
image_dir = root_dir + "photos/"
chunk_size = 20
chunks = divide_chunks(unlabeled_test, chunk_size)

paths  = []
failed = []
features = torch.zeros(1, 768)
for chunk in chunks:
    images = []
    for path in chunk:
        try:
            images.append(Image.open(image_dir + path))
            paths.append(path)
            print("\rLoading image %d of %d" % (len(paths) + len(failed), len(unlabeled_test)), end="")
        except Exception as e:
            print(e)
            print("\r                                  - Failed to load image %s" % path, end="")
            failed.append(path)
    print("\rProcessing image %d of %d" % (len(paths) + len(failed), len(unlabeled_test)), end="")
    inputs = processor(images=images, return_tensors="pt")
    image_features = model.get_image_features(**inputs)
    features = torch.cat((features, image_features), dim=0)

    # write data to file
    path_out    = "test/nocaption_paths.txt"
    feature_out = "test/nocaption_features.pt" # this is a binary file, shape is (n_images, 768)
    
    with open(path_out, "w") as f:
        for path in paths:
            f.write(path + "\n")
    
    with open(feature_out, "wb") as f:
        torch.save(features, f)

Processing image 100 of 100

In [17]:
# if continuing job
path_out    = "out/paths.txt"
feature_out = "out/features.pt" # this is a binary file, shape is (n_images, 768)
paths  = open(path_out).read().split("\n")
num_processed = len(paths) - 1
all_photos = all_photos[num_processed:]
image_dir = root_dir + "photos/"
chunk_size = 20
chunks = divide_chunks(all_photos, chunk_size)


failed = []
features = torch.load(feature_out)
for chunk in chunks:
    images = []
    for path in chunk:
        try:
            images.append(Image.open(image_dir + path))
            paths.append(path)
            print("\rLoading image %d of %d" % (len(paths) + len(failed), len(all_photos)), end="")
        except Exception as e:
            print(e)
            print("\r                                  - Failed to load image %s" % path, end="")
            failed.append(path)
    print("\rProcessing image %d of %d" % (len(paths) + len(failed), len(all_photos)), end="")
    inputs = processor(images=images, return_tensors="pt")
    image_features = model.get_image_features(**inputs)
    features = torch.cat((features, image_features), dim=0)

    write data to file
    with open(path_out, "w") as f:
        for path in paths:
            f.write(path + "\n")
    
    with open(feature_out, "wb") as f:
        torch.save(features, f)

SyntaxError: invalid syntax (1842262340.py, line 30)